# This notebook perform simulations of Cesium lead bromide (CsPbBr3)

# install various packages - change/leave as you like

In [ ]:
# conda install -c conda-forge numpy pandas matplotlib notebook mdtraj mdanalysis 
# pip install --upgrade --user ase
# or
# 
# git clone https://github.com/plumed/plumed2.git
# cd plumed2/
# plumed_dir=${PWD}
# #./configure --enable-modules=all #CC=mpiicc  CXX=mpiicpc  FC=mpiifort
# ./configure --enable-modules=all PYTHON_BIN=python3.9 #--prefix=/home/ahlawat/Downloads/softwares/plumed2/
# make -j 10
# make install
# source ${PWD}/sourceme.sh
# echo "source ${plumed_dir}/sourceme.sh" >> ~/.bashrc
# echo `export PYTHONPATH="/home/ahlawat/Downloads/softwares/plumed2/lib/plumed/python/:$PYTHONPATH"` >> ~/.bashrc
# #
# cd ../
# plumed="`pwd -P`/plumed2/src"

# wget http://lammps.sandia.gov/tars/lammps-stable.tar.gz
# tar xvf lammps-stable.tar.gz
# rm -rf lammps-stable.tar.gz
# mv `ls -d lammps-*` lammps
# cd lammps/src
# #replace this line manually or like this
# sed -i 's/liblink\/plumed\/src\/lib\/Plumed.inc/liblink\/Plumed.inc/g' ../lib/plumed/Install.py
# make lib-plumed args="-p $plumed -m shared"
# make yes-PLUMED
# make yes-DIFFRACTION
# make yes-RIGID
# make yes-KSPACE
# make yes-MOLECULE
# make yes-MANYBODY
# make yes-MOLECULE
# make yes-EXTRA-FIX
# make yes-EXTRA-DUMP
# make yes-MISC
# make -j 18 mpi
# mkdir bin
# mv lmp_mpi bin/
# lmp_path="`pwd -P` /bin/"
# echo `export PATH="lmp_path:$PATH"` >> ~/.bashrc

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ase.optimize.sciopt import *               
from ase.utils.geometry import *
from ase.lattice.spacegroup import crystal
from ase.visualize import view
from ase.lattice.surface import surface
from ase import Atoms
from ase.io import *
from ase.io import read, write
from ase.io.cif import read_cif
from ase.io.vasp import write_vasp
from abtem.visualize import show_atoms
from ase.io.lammpsdata import write_lammps_data
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate

import subprocess

import plumed
import nglview

/home/ahlawat/.local/lib/python3.9/site-packages/ase/utils/geometry.py:8: UserWarning: Moved to ase.geometry and ase.build
  warnings.warn('Moved to ase.geometry and ase.build')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/surface.py:17: UserWarning: Moved to ase.build
  warnings.warn('Moved to ase.build')


# building lammps data of a crystal file from experimental cif file

In [3]:
#import the unit cell cif file (experimental cif file)

unit_cell = read('cubic.cif')

#thick_z=6

# Create the surface exposing the 0 1 1 surface 

#cut = surface(unit_cell, (1,1,0),thick_z,vacuum=0)

#replicate the cell to obtain a supercell

rep1 = 6
rep2 = 6
rep3 = 6

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.75
    if(supercell.get_chemical_symbols()[i] == 'Br'):
        charge_array[i]= -0.75
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 1.50
    i = i + 1

supercell.set_initial_charges(charges=charge_array)

# view the supercell and obtain the supercell parameters (matrix)

view(supercell)
print(supercell) 
#print(supercell.get_cell())

write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

#fig, ax = plt.subplots(figsize=(8,5))
#plot_atoms(supercell, ax, radii=0.5, rotation=('90x,0y,0z'))
#ax.set_xlim(-20, 30)
#ax.set_xlabel(r'x[$\AA$]')
#ax.set_ylabel(r'y[$\AA$]')

write('new_cell.traj',supercell)

#view = nglview.show_ase(supercell)
#view.add_spacefill(radius_type='vdw', scale=0.1)
#view # display it


Atoms(symbols='Br648Cs216Pb216', pbc=True, cell=[35.244, 35.244, 35.244], initial_charges=..., spacegroup_kinds=...)


# viewers for geometries 

In [ ]:
# new_cell = read('new_cell.traj')
# from ase_notebook import AseView, ViewConfig

# ase_view = AseView(
#     rotations="45x,45y,45z",
#     atom_font_size=16,
#     axes_length=30,
#     canvas_size=(400, 400),
#     zoom=1.2,
#     show_bonds=True,
# )
# ase_view.config.uc_dash_pattern=(.6,.4)
# ase_view.add_miller_plane(1, 0, 0, color="green")

# ase_view.config.canvas_color_background = "blue"
# ase_view.config.canvas_background_opacity = 0.1

# gui = ase_view.make_render(new_cell)
# gui


# melt this crystal with very high temperature with NPT simulations with lammps

In [4]:
with open("start-melt.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           real
atom_style      full

variable        p_id world   1

variable        temperature equal 300.0
variable        temperature2 equal 1100.0
variable        tempDamp equal 100.0 # approx 0.1 ps

variable        pressure equal 1.00
variable        pressureDamp equal 500.0

variable        seed world 1428

read_data       data.CPI

mass            1 79.904     # Br
mass            2 132.904999 # Cs
mass            3 207.199997 # Pb

variable        freq equal 500

neighbor        0.3 bin
neigh_modify    check yes delay 0
kspace_style    pppm 1e-4         
dielectric      1.0

pair_style      lj/coul/long 10.0 10.0

variable ACs    equal "0.089856500"
variable rCs    equal "3.521013"

variable APb    equal "0.0051886219"
variable rPb    equal "4.043"

variable ABr    equal "0.026958651"
variable rBr    equal "4.82525"


variable APbBr  equal "0.507295"
variable rPbBr  equal "3.103"

variable ACsBr  equal "0.546757170"
variable rCsBr  equal "3.50408"


pair_coeff      1 1 ${ABr}    ${rBr}  
pair_coeff      2 2 ${ACs}    ${rCs}    
pair_coeff      3 3 ${APb}    ${rPb}

pair_coeff      1 2 ${ACsBr}  ${rCsBr} 
pair_coeff      1 3 ${APbBr}  ${rPbBr}
 


thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
restart         ${freq} restart.0 restart.2

# Minimization

minimize        1.0e-2 1.0e-3 100 1000

write_data      data.min

reset_timestep  0

# NVT

dump            myDump1 all atom 500 out.0.lammpstrj 

fix             1 all temp/csvr ${temperature} ${temperature2} ${tempDamp} ${seed}
fix             2 all nve

timestep        2.0

velocity        all create ${temperature} ${seed} dist gaussian
run             10000

unfix           1
unfix           2

write_data      data.NVT

undump          myDump1
reset_timestep  0

# NPT

dump            myDump2 all atom 500 out.1.lammpstrj 

fix             1 all temp/csvr ${temperature2} ${temperature2} ${tempDamp} ${seed}
fix             2 all nph iso ${pressure} ${pressure} ${pressureDamp} 
fix             3 all momentum 10000 linear 1 1 1

run             5000

unfix           1
unfix           2
unfix           3

undump          myDump2

reset_timestep  0

write_restart   restart.file
write_data      data.eq

""",file=f)


# run simulations 

In [5]:
subprocess.run("mpirun -np 30 lmp_intel_cpu_intelmpi < start-melt.lmp > log.lammps &",shell=True)

CompletedProcess(args='mpirun -np 30 lmp_intel_cpu_intelmpi < start-melt.lmp > log.lammps &', returncode=0)

In [6]:
traj = read('out.1.lammpstrj', index=":", parallel=True)
view(traj)

<Popen: returncode: None args: ['/home/ahlawat/miniconda3/bin/python', '-m',...>

In [11]:
# set point charges
i = 0
new_cell = traj[-1]
num_atoms = len(new_cell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(new_cell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.75
    if(new_cell.get_chemical_symbols()[i] == 'Br'):
        charge_array[i]= -0.75
    if(new_cell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 1.50
    i = i + 1

new_cell.set_initial_charges(charges=charge_array)

write_lammps_data('data_melt.CPI', new_cell, atom_style = 'full', force_skew=True, units='real')

In [20]:
new_cell

Atoms(symbols='H648He216Li216', pbc=True, cell=[45.27341189946993, 45.27341189946993, 45.27341189946993], initial_charges=...)

# lammps input file, ff are included 

In [21]:
with open("start-crystallization.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           real
atom_style      full

variable        p_id world   1

variable        temperature equal 1000.0
variable        temperature2 equal 600.0
variable        tempDamp equal 100.0 # approx 0.1 ps

variable        pressure equal 1.00
variable        pressureDamp equal 500.0

variable        seed world 1428

read_data       data_melt.CPI

mass            1 79.904     # Br
mass            2 132.904999 # Cs
mass            3 204.199997 # Pb


variable        freq equal 500

neighbor        0.3 bin
neigh_modify    check yes delay 0
kspace_style    pppm 1e-4         
dielectric      1.0


pair_style      lj/coul/long 10.0 10.0

variable ACs    equal "0.089856500"
variable rCs    equal "3.521013"

variable APb    equal "0.0051886219"
variable rPb    equal "4.043"

variable ABr    equal "0.026958651"
variable rBr    equal "4.82525"


variable APbBr  equal "0.507295"
variable rPbBr  equal "3.103"

variable ACsBr  equal "0.546757170"
variable rCsBr  equal "3.50408"


pair_coeff      1 1 ${ABr}    ${rBr}  
pair_coeff      2 2 ${ACs}    ${rCs}    
pair_coeff      3 3 ${APb}    ${rPb}

pair_coeff      1 2 ${ACsBr}  ${rCsBr} 
pair_coeff      1 3 ${APbBr}  ${rPbBr}



thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
restart         ${freq} restart.0 restart.2

# Minimization

minimize        1.0e-2 1.0e-3 100 1000

write_data      data.min

reset_timestep  0

# NVT

dump            myDump1 all atom 500 out.0.lammpstrj 

fix             1 all temp/csvr ${temperature} ${temperature2} ${tempDamp} ${seed}
fix             2 all nve

timestep        2.0

velocity        all create ${temperature} ${seed} dist gaussian
run             10000

unfix           1
unfix           2

write_data      data.NVT

undump          myDump1
reset_timestep  0

# NPT

dump            myDump2 all atom 500 out.1.lammpstrj 

fix             1 all plumed plumedfile plumed.dat outfile plumed.log
fix             2 all temp/csvr ${temperature2} ${temperature2} ${tempDamp} ${seed}
fix             3 all nph iso ${pressure} ${pressure} ${pressureDamp} 
fix             4 all momentum 10000 linear 1 1 1

run             1000000

unfix           1
unfix           2
unfix           3
unfix           4

undump          myDump2

reset_timestep  0

write_restart   restart.file
write_data      data.eq

""",file=f)



# run simulations

In [22]:
subprocess.run("nohup mpirun -np 30 lmp_intel_cpu_intelmpi < start-crystallization.lmp > log.lammps &",shell=True)

CompletedProcess(args='nohup mpirun -np 30 lmp_intel_cpu_intelmpi < start-crystallization.lmp > log.lammps &', returncode=0)

# see trajectories

In [23]:
traj = read('out.1.lammpstrj', index=":", parallel=True)
view(traj)

<Popen: returncode: None args: ['/home/ahlawat/miniconda3/bin/python', '-m',...>

# plotting plumed outputs

In [ ]:
colvar=plumed.read_as_pandas("COLVAR")
plt.plot(colvar.time,colvar.c1,"x",label="c1")
plt.plot(colvar.time,colvar.c2,"x",label="c2")

plt.xlabel("time")
plt.ylabel("colvar")
plt.legend()
plt.show()

plt.plot(colvar.time,colvar.m,"x",label="m")
plt.xlabel("time")
plt.ylabel("m")
plt.legend()
plt.show()

plt.plot(colvar.c1,colvar.c2,"x")
plt.xlabel("c1")
plt.ylabel("c2")
plt.show()

plt.plot(colvar.c1-colvar.c2,"x")
plt.show()
